# import module

In [9]:
import mediapipe as mp
import numpy as np
import time
import cv2
import torch
import math
import pandas as pd
import os
import time

import utils
import augmentation


device = torch.device("cpu")
# import gensim

# Data Load

In [5]:

num_csv1 = pd.read_csv("./output/csv/KETI-2017-SL-0_10480-v2_1_num.csv")
num_csv2 = pd.read_csv("./output/csv/KETI-2018-SL-10480_43492-v1_nums.csv")

num_csv1 = pd.concat([num_csv1,num_csv2])


ipykernel_launcher:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [6]:
dir_names = num_csv1['파일명']
kors=num_csv1['한국어']

In [7]:
def pipe2(tensor_list, nums, batch_first = False, device = "cuda"):
    '''
    15frame 기준으로 해서 
    25개의 데이터와 10개의 겹친 데이터로 변경

    '''
    new_tensor = []
    index = []
    i=0

    for tensor in tensor_list:
        size= tensor.shape[0]//50
        for x in range(size-1):
            new_tensor.append(tensor[x*50:(x+1)*50+20:2])
            index.append(nums[i])
            ## 남은 개수 50~90
        if tensor.shape[0] - size*50 > 20:
            new_tensor.append(tensor[(size-1)*50:(size)*50+20:2])
            index.append(nums[i])
        new_tensor.append(tensor[(size)*50 ::2])
        index.append(nums[i])
        
        i+=1
    # print(new_tensor[0].shape)
    tensor_list = torch.nn.utils.rnn.pad_sequence(list(map(
            lambda x: x.view(-1,x.shape[1]*x.shape[2]),new_tensor)),
            batch_first=batch_first
            ).to(device)
    
    return  tensor_list, index


In [10]:
hand, nums = utils.load_tensor_hand(dir_names)

In [12]:
hand_tensor, hand_index = pipe2(hand,nums,device)

IndexError: list index out of range

In [13]:
class PositionalEncoding(torch.nn.Module):

    def __init__(self, d_model,batch_first = False, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = torch.nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        # print(pe.shape)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)        
        # print(pe.shape)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.batch_first = batch_first
        if batch_first :
            pe = pe.unsqueeze(0)
        else :
            pe = pe.unsqueeze(0).transpose(0,1)
        # print(pe.shape)
        self.register_buffer('pe', pe)

    def forward(self, x):

        if self.batch_first:
            # print(x.shape , self.pe[:,:x.size(1),:].shape)
            x = x + self.pe[:, :x.size(1),:]
        else :
            # print(x.shape , self.pe[:x.size(0),:].shape)
            x = x + self.pe[:x.size(0),:]
            
        return self.dropout(x)

In [ ]:
answer = kors[hand_index]
print(answer.shape, answer.unique().size)


In [ ]:
# tokenizer, pad_sequence
sos_data = torch.LongTensor(np.zeros(answer.size)+answer.unique().size).unsqueeze(dim=0)
decode_data = torch.LongTensor(np.array(answer)).unsqueeze(dim=0)
decode_data = torch.cat((sos_data,decode_data),axis=0)

In [ ]:
decode_data[:,-10:], sos_data.shape , decode_data.shape 
# seq, batch

In [ ]:
eos_y_data = torch.zeros(1,answer.size,answer.unique().size+2)
# (1, 153, 53)
y_data = torch.zeros(1,answer.size,answer.unique().size+2)
# (1, 153 , 53)

y_data = torch.cat((y_data,eos_y_data),axis=0).to(device)
y_data[0,torch.arange(answer.size),np.array(answer)] = 1 
#one hot vector

y_data[1,torch.arange(answer.size),-1] = 1 #eos

In [ ]:
y_data.shape, decode_data[:,10] , y_data[:,10] 
# (sos, answer)   , (answer , eos)
# sos = 51 , eos = 52

In [16]:
class TransformerModel_hand(torch.nn.Module):

    def __init__(self, input_node, d_model, nhead, nhid, nlayers,ntoken ,dropout=0.5  ):
        """
        input_node  node개수
        d_model    임배딩 차원 , model 내부의 수치
        nhead   멀티헤드 개수
        nhid    Linear 뉴런개수 = dff 
        nlayers encoder 개수 = num_layers
        ntoken  단어 개수
        """
        super(TransformerModel_hand, self).__init__()
        self.model_type = 'Transformer'
        self.d_model = d_model

        self.encode_emb = torch.nn.Linear(input_node,d_model)# GNN으로 대체될수도 있음
        self.decode_emb = torch.nn.Embedding(ntoken,d_model)

        self.pos_encoder = PositionalEncoding(d_model, dropout) #해당하는 언어의 위치 정보

        self.transformer = torch.nn.Transformer(
                d_model = d_model, nhead = nhead, 
                num_encoder_layers = nlayers, 
                num_decoder_layers = nlayers, 
                dim_feedforward = nhid, dropout = dropout
            )

        self.decoder = torch.nn.Linear(d_model, ntoken)
        self.softmax = torch.nn.Softmax(dim=2)

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask


    def forward(self, src ,tgt ,src_mask = None, tgt_mask = None):
        # src = torch.cat(src,axis=2)
        # lt, rt, pt 의 feature부분을 concat
        src = self.encode_emb(src)
        src = self.pos_encoder(src)

        tgt = self.decode_emb(tgt)
        tgt = self.pos_encoder(tgt)

        # print("source shape : {0} | target shape : {1}".format(src.shape,tgt.shape))
        target = self.transformer(src=src,tgt=tgt,src_mask = None, tgt_mask = None)
        output = self.decoder(target)
        # output = self.softmax(output)
        return output

In [18]:
nodes = 126 # GNN이전의 노드 개수  고친다면
d_model = 400 #400
nlayers = 4 
nhead = 8
dropout = 0.2
nhid = 2048
tokens = answer.unique().size+2# token개수 + sos, eos
transmodel = TransformerModel_hand(input_node = nodes,d_model= d_model,nhead = nhead,nhid=nhid,nlayers=nlayers,ntoken=tokens,dropout=dropout).to(device)

loss_fn =  torch.nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.Adadelta(transmodel.parameters())

NameError: name 'TransformerModel' is not defined

In [ ]:
src_mask = transmodel.generate_square_subsequent_mask(hand_tensor.shape[0]).to(device)
# src mask의 경우 다른 mask를 사용해야 하는것으로 판단됨
tgt_mask = transmodel.generate_square_subsequent_mask(decode_data.shape[0]).to(device)

In [ ]:
x = 0
batch_size = 40
tmp = int(lt_list.shape[1]/batch_size)
src =  (hand_tensor[:,x*batch_size:(x+1)*batch_size]) 

In [ ]:
aug = DataAugmentation(src,0.5,1,device)

In [ ]:

epochs = 1000
losses = []
accuracy = []
transmodel.train()
start = time.time()
for i in range(epochs):
    tmp_time = time.time()
    print("start epoch ",end="| ")
    # print(tmp)
    batch_accuracy = 0
    batch_loss = 0
    for x in range(tmp):
        # print(src[0].shape, tgt.shape)
        # print("{0} {1:.3f}s ".format(x,time.time()-tmp_time),end="")
        src = ( aug(lt_list[:,x*batch_size:(x+1)*batch_size]), 
                aug(rt_list[:,x*batch_size:(x+1)*batch_size]),
                pt_list[:,x*batch_size:(x+1)*batch_size]
                )
        tgt = decode_data[0,x*batch_size:(x+1)*batch_size].unsqueeze(dim=0)
        y = y_data[0,x*batch_size:(x+1)*batch_size].unsqueeze(dim=0)


        optimizer.zero_grad()

        predict = transmodel(src,tgt, src_mask )
        
        tmp_pre = predict.squeeze(0).max(dim=-1, keepdim=False)[1]
        tmp_y = y.squeeze(0).max(dim=-1, keepdim=False)[1]
        
        # predict (seq =1 , batch = 30, feature =  56)
        
        predict = predict.transpose(0,1)
        y = y.transpose(0,1)
        
        loss =loss_fn(predict,y)
        # batch, **
        
        loss.backward()
        optimizer.step()


        batch_loss += loss.item()
        batch_accuracy +=  batch_size-(tmp_pre - tmp_y).count_nonzero()
#         break
    # print(batch_loss, batch_accuracy)
    losses.append(batch_loss)
    accuracy.append(batch_accuracy/(batch_size*tmp))
    
    end = time.time()
    # print(predict.shape,predict.squeeze(0).max(dim=-1, keepdim=False)[1],end=" ")
    print("| end epoch: {0}\t| time: {2:.4f}s |  loss : {1:6f} | acc : {3:.2f}%".format(i+1,losses[-1], end-tmp_time, accuracy[-1]*100))   
#     if(losses[-1] < 0.5 and accuracy[-1] > 0.97):
#         break
print("total time = {0:.4f}s".format(end-start))

        

In [ ]:
src = hand_tensor[:,batch_size*tmp:]
tgt = decode_data[0,batch_size*tmp:].unsqueeze(dim=0)
y = y_data[0,batch_size*tmp:].unsqueeze(dim=0).transpose(0,1)


In [ ]:
src.shape, tgt.shape, y.shape

In [ ]:
val_predict = transmodel(src,tgt,src_mask).transpose(0,1)

In [ ]:
val_loss = loss_fn(val_predict, y)

In [ ]:
val_loss.item()

In [ ]:
 val_predict.transpose(0,1).squeeze(0).max(dim=-1, keepdim=False)[1] , tgt , y.transpose(0,1).squeeze(0).max(dim=-1, keepdim=False)[1]

In [ ]:
val_predict[0,0]

In [ ]:
t = [(i,x.item()) for i,x in enumerate(val_predict[0,0])]
t = sorted(t , key=lambda x:-x[1] )
t